In [ ]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline

**Learn the Basics** \|\| [Quickstart](quickstart_tutorial.html) \|\|
[Tensors](tensorqs_tutorial.html) \|\| [Datasets &
DataLoaders](data_tutorial.html) \|\|
[Transforms](transforms_tutorial.html) \|\| [Build
Model](buildmodel_tutorial.html) \|\|
[Autograd](autogradqs_tutorial.html) \|\|
[Optimization](optimization_tutorial.html) \|\| [Save & Load
Model](saveloadrun_tutorial.html)

Learn the Basics
================

Authors: [Suraj Subramanian](https://github.com/subramen), [Seth
Juarez](https://github.com/sethjuarez/), [Cassie
Breviu](https://github.com/cassiebreviu/), [Dmitry
Soshnikov](https://soshnikov.com/), [Ari
Bornstein](https://github.com/aribornstein/)

Most machine learning workflows involve working with data, creating
models, optimizing model parameters, and saving the trained models. This
tutorial introduces you to a complete ML workflow implemented in
PyTorch, with links to learn more about each of these concepts.

We\'ll use the FashionMNIST dataset to train a neural network that
predicts if an input image belongs to one of the following classes:
T-shirt/top, Trouser, Pullover, Dress, Coat, Sandal, Shirt, Sneaker,
Bag, or Ankle boot.

[This tutorial assumes a basic familiarity with Python and Deep Learning
concepts.]{.title-ref}

Running the Tutorial Code
-------------------------

You can run this tutorial in a couple of ways:

-   **In the cloud**: This is the easiest way to get started! Each
    section has a \"Run in Microsoft Learn\" and \"Run in Google Colab\"
    link at the top, which opens an integrated notebook in Microsoft
    Learn or Google Colab, respectively, with the code in a fully-hosted
    environment.
-   **Locally**: This option requires you to setup PyTorch and
    TorchVision first on your local machine ([installation
    instructions](https://pytorch.org/get-started/locally/)). Download
    the notebook or copy the code into your favorite IDE.

How to Use this Guide
---------------------

If you\'re familiar with other deep learning frameworks, check out the
[0. Quickstart](quickstart_tutorial.html) first to quickly familiarize
yourself with PyTorch\'s API.

If you\'re new to deep learning frameworks, head right into the first
section of our step-by-step guide: [1. Tensors](tensor_tutorial.html).

::: {.toctree maxdepth="2" hidden=""}
quickstart\_tutorial tensorqs\_tutorial data\_tutorial
transforms\_tutorial buildmodel\_tutorial autogradqs\_tutorial
optimization\_tutorial saveloadrun\_tutorial
:::


In [54]:
import torch
tn=torch.ones(2,3)
torch.sqrt(tn)
torch.sum(tn).item()
torch.mean(tn)
torch.max(tn)
tn @ tn.T
tn * tn
torch.exp(tn)
torch.sin(tn)
torch.matmul(tn, tn.T)
torch.eq(tn, tn)
torch.where(tn<0.5,1,2)
tn.add_(2)
tn
torch.matmul(tn, tn.T)
tn.view(6)
tn[:,1]
tn.reshape(3,2)
torch.cat((tn,tn), dim=0)
torch.cat((tn,tn), dim=1)
torch.split(tn,2)

x=torch.ones(3,3, requires_grad=True)
print(x)
y=x+2
z=y*y*3
out=z.mean()
out
out.backward()
x.grad

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], requires_grad=True)


tensor([[2., 2., 2.],
        [2., 2., 2.],
        [2., 2., 2.]])

In [84]:
import torch
import torch.nn as nn
import torch.optim as optim
model = nn.Linear(10, 1)
criterian = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)
# dir(nn)
inputs = torch.rand(64,10)
targets = torch.rand(64, 1)

outputs = model(inputs)
loss = criterian(outputs, targets)
loss.backward()
optimizer.step()
optimizer.zero_grad()

outputs

tensor([[-0.1112],
        [ 0.1307],
        [ 0.2470],
        [-0.0829],
        [ 0.0377],
        [-0.0230],
        [-0.0785],
        [ 0.0865],
        [ 0.1348],
        [ 0.0619],
        [-0.1859],
        [ 0.0767],
        [ 0.0081],
        [-0.2460],
        [-0.0201],
        [-0.1682],
        [-0.1256],
        [ 0.0580],
        [-0.0377],
        [ 0.0178],
        [ 0.1653],
        [-0.0375],
        [-0.2233],
        [ 0.2875],
        [ 0.1800],
        [-0.1073],
        [ 0.2486],
        [-0.1675],
        [-0.3484],
        [-0.2581],
        [ 0.2007],
        [-0.2445],
        [-0.0410],
        [-0.1027],
        [-0.1849],
        [ 0.0016],
        [ 0.0979],
        [-0.3675],
        [ 0.1467],
        [-0.0642],
        [ 0.0560],
        [-0.2267],
        [-0.1414],
        [-0.2704],
        [ 0.1008],
        [-0.1734],
        [-0.0107],
        [ 0.2302],
        [-0.3435],
        [-0.2089],
        [-0.1362],
        [-0.0285],
        [-0.

In [90]:
import torch
import torch.nn as nn


class SimpleNetwork(nn.Module):
  def __init__(self):
    super(SimpleNetwork, self).__init__()
    self.fc1 = nn.Linear(10, 5)
    self.fc2 = nn.Linear(5, 3)

  def forward(self, x):
    x = torch.relu(self.fc1(x))
    return self.fc2(x)

model = SimpleNetwork()
x = torch.randn(1, 10, requires_grad=True)
output = model(x)
output

tensor([[ 0.2822, -0.1459, -0.4327]], grad_fn=<AddmmBackward0>)

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms

transform = transforms.ToTensor()
train_data = datasets.MNIST(
    root='data',
    train=True,
    transform=transform,
    download=True
)
test_data = datasets.MNIST(
    root='data',
    train=False,
    transform=transform,
    download=True
)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=1000, shuffle=False)

class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
    self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
    self.fc1 = nn.Linear(320, 50)
    self.fc2 = nn.Linear(50, 10)

  def forward(self, x):
    x = F.relu(F.max_pool2d(self.conv1(x), 2))
    x = F.relu(F.max_pool2d(self.conv2(x), 2))
    x = x.view(-1, 320)
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return F.log_softmax(x, dim=1)

device = torch.device("cpu" if torch.cuda.is_available else "cpu")
model = CNN().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

for epoch in range(1, 6):
  model.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    data, target = data.to(device), target.to(device)
    optimizer.zero_grad()
    output = model(data)
    loss = loss_fn(output, target)
    loss.backward()
    optimizer.step()
  print("Epoch {epoch} complete")

model.eval()
correct = 0
with torch.no_grad():
  for data, target in test_loader:
    data, target = data.to(device), target.to(device)
    output = model(data)
    pred = output.argmax(dim=1)
    correct += pred.eq(target).sum().item()
print(f"Test accuracy {correct/len(test_loader.dataset):.2%}")



#dir(nn)
#help(nn.Conv2d)

Epoch {epoch} complete
Epoch {epoch} complete
Epoch {epoch} complete
Epoch {epoch} complete
Epoch {epoch} complete
Test accuracy 98.70%
